***2.0. Import Libraries***

In [1]:
!pip install torchmetrics

import gc
import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.nn.functional as F
from keras.datasets import fashion_mnist
from torch.utils.data import DataLoader, Dataset
from torchmetrics.classification import BinaryF1Score
from torchmetrics.classification import BinaryAccuracy
from torchmetrics.classification import BinaryPrecision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


*Clear Cache*

In [2]:
torch.cuda.empty_cache()
gc.collect()
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |

**2.1. Load Fashion_MNIST Datasets**

In [3]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)

x_test = torch.tensor(x_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

*Create Data Class*

In [4]:
class My_Data():
  def __init__(self, x, y, k):
    self.x = x
    self.z = np.zeros((self.x.shape[0], k))
    for i in range(self.x.shape[0]):
      self.z[i][int(y[i])] = 1 

  def __len__(self):
    return self.x.shape[0]

  def __getitem__(self, index):
    return self.x[index], self.z[index]

**2.2. Choose tow Architecture (2th, 3th, & 4th Architecture)**

In [5]:
class CNN_Arc2(nn.Module):
  def __init__(self, K):
    super(CNN_Arc2, self).__init__()

    self.conv1 = nn.Conv2d(1, 64, kernel_size = 2)
    self.conv2 = nn.Conv2d(64, 64, kernel_size = 2)
    self.maxpool = nn.MaxPool2d((2, 2))
    self.dropout = nn.Dropout(p = 0.25)
    self.linear1 = nn.Linear(9216, 64)
    self.linear2 = nn.Linear(64, 10)

  def forward(self, x):
    x = F.softmax(self.conv1(x), dim = 1)
    x = self.maxpool(x)
    x = self.dropout(x)
    
    x = F.softmax(self.conv2(x), dim = 1)
    x = self.dropout(x)

    x = x.view(x.size(0), -1)
    x=self.linear1(x)
    x = self.dropout(x)

    x = self.linear2(x)
    return x

In [6]:
class CNN_Arc3(nn.Module):
  def __init__(self, K):
    super(CNN_Arc3, self).__init__()

    self.conv1 = nn.Conv2d(1, 64, kernel_size = 2)
    self.conv2 = nn.Conv2d(64, 64, kernel_size = 2)
    self.maxpool = nn.MaxPool2d((2, 2))
    self.dropout = nn.Dropout(p = 0.1)
    self.linear1 = nn.Linear(1600,64)
    self.linear2 = nn.Linear(64,10)

  def forward(self, x):
    x = F.softmax(self.conv1(x), dim = 1)
    x = self.maxpool(x)
    x = self.dropout(x)
    
    x = F.softmax(self.conv2(x), dim = 1)
    x = self.maxpool(x)
    x = self.dropout(x)

    x = F.softmax(self.conv2(x), dim = 1)
    x = self.dropout(x)

    x = x.view(x.size(0), -1)
    x = self.linear1(x)
    x = self.dropout(x)

    x = self.linear2(x)
    return x

In [7]:
class CNN_Arc4(nn.Module):
  def __init__(self, K):
    super(CNN_Arc4, self).__init__()

    self.conv1 = nn.Conv2d(1, 64, kernel_size = 2)
    self.conv2 = nn.Conv2d(64, 64, kernel_size = 2)
    self.maxpool = nn.MaxPool2d((2, 2))
    self.dropout = nn.Dropout(p = 0.1)
    self.linear1 = nn.Linear(64,64)
    self.linear2 = nn.Linear(64,10)

  def forward(self, x):
    x = F.softmax(self.conv1(x), dim = 1)
    x = self.maxpool(x)
    x = self.dropout(x)
    
    x = F.softmax(self.conv2(x), dim = 1)
    x = self.maxpool(x)
    x = self.dropout(x)

    x = F.softmax(self.conv2(x), dim = 1)
    x = self.maxpool(x)
    x = self.dropout(x)

    x = F.softmax(self.conv2(x), dim = 1)
    x = self.dropout(x)

    x = x.view(x.size(0), -1)
    x = self.linear1(x)
    x = self.dropout(x)

    x = self.linear2(x)
    return x

*Load and batch Train Data and Test Data*

In [8]:
train_data = My_Data(x_train, y_train, 10)
test_data = My_Data(x_test, y_test, 10)

train = torch.utils.data.DataLoader(train_data, 128, True)
test = torch.utils.data.DataLoader(test_data, 128, False)

*Use Architecture*

In [9]:
k = len(torch.unique(y_train))
Fashion_MNIST = CNN_Arc4(k)
epochs_number = 50

*Create Optimizer, Loss Function, f1Score, Accuracy, and Precision*

In [10]:
optimizer = torch.optim.SGD(Fashion_MNIST.parameters(), lr = 0.01)
loss_function = nn.CrossEntropyLoss()

f1sc = BinaryF1Score()
acc = BinaryAccuracy()
pre = BinaryPrecision()
softmax = nn.Softmax(dim = 1)

f1score = []
accuracy = []
precision = []

*Create Device and Add Network and ... to Device*

In [11]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
loss_function=loss_function.to(device)
Fashion_MNIST.to(device)
acc.to(device)
pre.to(device)
f1sc.to(device)

BinaryF1Score()

*Train and Test CNN*

In [12]:
for epoch in range(epochs_number):
  train_loss = []
  test_loss = []
  Fashion_MNIST.train()
  for data, pred in train:
    data = data.unsqueeze(1)
    data = data.to(device)
    pred = pred.to(device)
    optimizer.zero_grad()
    output = Fashion_MNIST(data)
    loss = loss_function(output.to(device), pred)
    loss.backward()
    train_loss.append(loss.item())
    optimizer.step()
  
  Fashion_MNIST.eval()
  with torch.no_grad():
    for data, pred in test:
      data = data.unsqueeze(1)
      data = data.to(device)
      pred = pred.to(device)
      outputs = Fashion_MNIST(data)
      outputs = softmax(outputs)
      los = loss_function(outputs.to(device), pred)
      test_loss.append(los.item())
      accuracy.append(acc(outputs, pred))
      precision.append(pre(outputs, pred))
      f1score.append(f1sc(outputs, pred))
  print(f'epoch {epoch} : Accuracy {sum(accuracy)/len(accuracy)*100}   Precision {sum(precision)/len(precision)*100}   f1score {sum(f1score)/len(f1score)*100}')



epoch 0 : Accuracy 90.00006866455078   Precision 0.0   f1score 0.0
epoch 1 : Accuracy 90.00003814697266   Precision 0.0   f1score 0.0
epoch 2 : Accuracy 89.99981689453125   Precision 0.0   f1score 0.0
epoch 3 : Accuracy 89.99971771240234   Precision 0.0   f1score 0.0
epoch 4 : Accuracy 89.99964904785156   Precision 0.0   f1score 0.0
epoch 5 : Accuracy 89.99960327148438   Precision 0.0   f1score 0.0
epoch 6 : Accuracy 89.99958038330078   Precision 0.0   f1score 0.0
epoch 7 : Accuracy 89.99986267089844   Precision 0.0   f1score 0.0
epoch 8 : Accuracy 90.0001449584961   Precision 0.0   f1score 0.0
epoch 9 : Accuracy 90.00038146972656   Precision 0.0   f1score 0.0
epoch 10 : Accuracy 90.00056457519531   Precision 0.0   f1score 0.0
epoch 11 : Accuracy 90.00071716308594   Precision 0.0   f1score 0.0
epoch 12 : Accuracy 90.0008544921875   Precision 0.0   f1score 0.0
epoch 13 : Accuracy 90.00096893310547   Precision 0.0   f1score 0.0
epoch 14 : Accuracy 90.00106811523438   Precision 0.0   f1sc